need files
- dataset(zip)
- latent(npy)

<b>After SetUP, these files <font color = "red">must be </font>uploaded and <font color = "red">DO </font> daijinakoto</b>

#####SetUP

In [ ]:
# @title Install
#!huggingface-cli login
from google.colab import files
#files.upload()


In [ ]:
# @title Install Library
!pip install peft datasets transformers diffusers trimesh pynvml
#!pip install torch_xla
!pip install -U fpsample
!git clone https://github.com/openai/shap-e.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.6/707.6 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# @title Load&Setting
Device = "GPU" # @param ["GPU","TPU"]
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
import torch
from datasets import load_dataset
import os
#import torch_xla.core.xla_model as xm

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from datasets import load_dataset

import torch


if Device == "GPU":
    device = "cuda"
# elif Device == "TPU":
#     device = xm.xla_device()

model_name_or_path = "openai/shap-e"
#tokenizer_name_or_path = "openai/clip-vit-base-patch16"

checkpoint_name = "shap-e_lora_v1.pt"
text_column = "text"
# label_column = "text_label"
max_length = 128



##### daijinakoto

/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/shap_e/renderer.py
に下記コードを追加する
```python
    def new_mesh(
        self,
        latents,
        device,
        grid_size: int = 128,
        query_batch_size: int = 4096,
        texture_channels: Tuple = ("R", "G", "B"),
    ):
        # 1. project the parameters from the generated latents
        projected_params = self.params_proj(latents)

        #change key

        # 新しいキーの辞書を作成
        # updated_projected_params = {}

        # for key, value in projected_params.items():
        #     # nerstf.mlp.X.weight のパターンを探す
        #     if key.startswith("nerstf.mlp.") and key.endswith(".weight"):
        #         # 新しいキーに変換
        #         updated_key = key.replace(".weight", ".base_layer.weight")
        #         updated_projected_params[updated_key] = value
        #     else:
        #         # それ以外のキーはそのまま保持
        #         updated_projected_params[key] = value

        # # 更新された辞書に置き換え
        # projected_params = updated_projected_params

        # 確認
        # for key in projected_params.keys():
        #     print(f"Updated key: {key}")

        # for key, value in projected_params.items():
        #     print(f"Key: {key}, Mean: {value.mean().item()}, Std: {value.std().item()}")

        # for name, param in self.mlp.state_dict().items():
        #     print(f"{name}: mean={param.mean().item()}, std={param.std().item()}")

        # 2. update the mlp layers of the renderer
        for name, param in self.mlp.state_dict().items():
            if f"nerstf.{name}" in projected_params.keys():
                param.copy_(projected_params[f"nerstf.{name}"].squeeze(0))
                

        # for name, param in self.mlp.state_dict().items():
        #     print(f"{name}: mean={param.mean().item()}, std={param.std().item()}")
        # 3. decoding with STF rendering
        # 3.1 query the SDF values at vertices along a regular 128**3 grid

        query_points = volume_query_points(self.volume, grid_size)
        query_positions = query_points[None].repeat(1, 1, 1).to(device=device, dtype=self.mlp.dtype).requires_grad_(True)

        fields = []

        for idx in range(0, query_positions.shape[1], query_batch_size):
            query_batch = query_positions[:, idx : idx + query_batch_size]

            model_out = self.mlp(
                position=query_batch, direction=None, ts=None, nerf_level="fine", rendering_mode="stf"
            )
            fields.append(model_out.signed_distance)

        # predicted SDF values
        fields = torch.cat(fields, dim=1)
        fields = fields.float()

        assert (
            len(fields.shape) == 3 and fields.shape[-1] == 1
        ), f"expected [meta_batch x inner_batch] SDF results, but got {fields.shape}"

        fields = fields.reshape(1, *([grid_size] * 3))

        # create grid 128 x 128 x 128
        # - force a negative border around the SDFs to close off all the models.
        full_grid = torch.zeros(
            1,
            grid_size + 2,
            grid_size + 2,
            grid_size + 2,
            device=fields.device,
            dtype=fields.dtype,
        )
        full_grid.fill_(-1.0)
        full_grid[:, 1:-1, 1:-1, 1:-1] = fields
        fields = full_grid

        # apply a differentiable implementation of Marching Cubes to construct meshs
        raw_meshes = []
        mesh_mask = []

        for field in fields:
            raw_mesh = self.mesh_decoder(field, self.volume.bbox_min, self.volume.bbox_max - self.volume.bbox_min)
            mesh_mask.append(True)
            raw_meshes.append(raw_mesh)

        mesh_mask = torch.tensor(mesh_mask, device=fields.device)
        max_vertices = max(len(m.verts) for m in raw_meshes)

        # 3.2. query the texture color head at each vertex of the resulting mesh.
        texture_query_positions = torch.stack(
            [m.verts[torch.arange(0, max_vertices) % len(m.verts)] for m in raw_meshes],
            dim=0,
        ).requires_grad_(True)
        # print("quert_pos_shape",texture_query_positions.shape)
        texture_query_positions = texture_query_positions.to(device=device, dtype=self.mlp.dtype)

        textures = []


        for idx in range(0, texture_query_positions.shape[1], query_batch_size):
            query_batch = texture_query_positions[:, idx : idx + query_batch_size]

            texture_model_out = self.mlp(
                position=query_batch, direction=None, ts=None, nerf_level="fine", rendering_mode="stf"
            )
            textures.append(texture_model_out.channels)
            # print(f"texture_model_out.channels: {texture_model_out.channels}")

        # predict texture color
        textures = torch.cat(textures, dim=1)

        textures = _convert_srgb_to_linear(textures)
        textures = textures.float()

        # 3.3 augument the mesh with texture data
        assert len(textures.shape) == 3 and textures.shape[-1] == len(
            texture_channels
        ), f"expected [meta_batch x inner_batch x texture_channels] field results, but got {textures.shape}"

        for m, texture in zip(raw_meshes, textures):
            texture = texture[: len(m.verts)]
            m.vertex_channels = dict(zip(texture_channels, texture.unbind(-1)))

        return raw_meshes[0]

```



##### caution
<b> DONT run before finish upload

#####Loading

In [ ]:
# @title LoadModel
from diffusers import ShapEPipeline

# 事前学習済みモデル名
model_name = "openai/shap-e"

# 1. ShapEPipeline をロード
pipeline = ShapEPipeline.from_pretrained(model_name)

# 2. モデルを取得
model = pipeline# Renderer モデル
print("********model********")
print(model)
print("******renderer*******")
model.shap_e_renderer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/905M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/905M [00:00<?, ?B/s]

...

The config attributes {'renderer': ['shap_e', 'ShapERenderer']} were passed to ShapEPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'renderer': ['shap_e', 'ShapERenderer']} are not expected by ShapEPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--openai--shap-e/snapshots/7bd337afdea1c17842e1c3cc45c4e268356dba40/prior: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--openai--shap-e/snapshots/7bd337afdea1c17842e1c3cc45c4e268356dba40/prior.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--openai--shap-e/snapshots/7bd337afdea1c17842e1c3cc45c4e268356dba40/shap_e_renderer: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--openai--shap-e/snapshots/7bd337afdea1c17842e1c3cc45c4e268356dba40/shap_e_renderer.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


********model********
ShapEPipeline {
  "_class_name": "ShapEPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "openai/shap-e",
  "prior": [
    "diffusers",
    "PriorTransformer"
  ],
  "scheduler": [
    "diffusers",
    "HeunDiscreteScheduler"
  ],
  "shap_e_renderer": [
    "shap_e",
    "ShapERenderer"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ]
}

******renderer*******


ShapERenderer(
  (params_proj): ShapEParamsProjModel(
    (projections): ModuleDict(
      (nerstf__mlp__0__weight): ChannelsProj(
        (proj): Linear(in_features=1024, out_features=23808, bias=True)
        (norm): LayerNorm((93,), eps=1e-05, elementwise_affine=True)
      )
      (nerstf__mlp__1__weight): ChannelsProj(
        (proj): Linear(in_features=1024, out_features=65536, bias=True)
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      )
      (nerstf__mlp__2__weight): ChannelsProj(
        (proj): Linear(in_features=1024, out_features=65536, bias=True)
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      )
      (nerstf__mlp__3__weight): ChannelsProj(
        (proj): Linear(in_features=1024, out_features=65536, bias=True)
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (mlp): MLPNeRSTFModel(
    (mlp): ModuleList(
      (0): Linear(in_features=93, out_features=256, bias=True)
      (1-3)

In [ ]:
# @title Adapt LoRA
target_modules = "mlp,proj(手動で変える)" # @param {"type":"string","placeholder":"proj"}
r = 4 # @param {"type":"number","placeholder":"8"}
lora_alpha = 32 # @param {"type":"number","placeholder":"lora_alpha"}
lora_dropout = 0.1 # @param {"type":"number","placeholder":"lora_dropout"}


from transformers import AutoModelForPreTraining
import torch.nn.init as init

# creating model
peft_config = LoraConfig(
    target_modules = ["proj","mlp.4","mlp.5","mlp.6"],
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
    )
model.shap_e_renderer = get_peft_model(model.shap_e_renderer, peft_config)
for name, param in model.shap_e_renderer.named_parameters():
    if "lora_B" in name:
        print(f"{name}: mean={param.mean().item()}, std={param.std().item()}")



model.shap_e_renderer.print_trainable_parameters()
model.shap_e_renderer

base_model.model.params_proj.projections.nerstf__mlp__0__weight.proj.lora_B.default.weight: mean=0.0, std=0.0
base_model.model.params_proj.projections.nerstf__mlp__1__weight.proj.lora_B.default.weight: mean=0.0, std=0.0
base_model.model.params_proj.projections.nerstf__mlp__2__weight.proj.lora_B.default.weight: mean=0.0, std=0.0
base_model.model.params_proj.projections.nerstf__mlp__3__weight.proj.lora_B.default.weight: mean=0.0, std=0.0
base_model.model.mlp.mlp.4.lora_B.default.weight: mean=0.0, std=0.0
base_model.model.mlp.mlp.5.lora_B.default.weight: mean=0.0, std=0.0
base_model.model.mlp.mlp.6.lora_B.default.weight: mean=0.0, std=0.0
trainable params: 903,420 || all params: 227,200,706 || trainable%: 0.3976


PeftModel(
  (base_model): LoraModel(
    (model): ShapERenderer(
      (params_proj): ShapEParamsProjModel(
        (projections): ModuleDict(
          (nerstf__mlp__0__weight): ChannelsProj(
            (proj): lora.Linear(
              (base_layer): Linear(in_features=1024, out_features=23808, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=4, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=4, out_features=23808, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict()
            )
            (norm): LayerNorm((93,), eps=1e-05, elementwise_affine=True)
          )
          (nerstf__mlp__1__weight): ChannelsProj(
        

In [ ]:
# @title Dataset Upload

import zipfile
import os

# アップロードした ZIP ファイルの名前を指定
file_name = "Dataset" # @param {"type":"string","placeholder":"dataaset.zip"}
zip_file_name = file_name + ".zip"
# 解凍先ディレクトリ
extract_dir = "/content/extracted_folder"

# 解凍処理
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")

Files extracted to: /content/extracted_folder


In [ ]:
# @title modelID取得
import os
import json

# 基準となるディレクトリのパス
base_directory = "/content/extracted_folder/"

# IDを格納するリスト
lines = []

# ディレクトリ内を再帰的に検索
for root, _, files in os.walk(base_directory):
    for f in files:
        # "model_normalized.json"のみ対象
        if f == "model_normalized.json":
            file_path = os.path.join(root, f)
            try:
                # JSONファイルを開いて読み込む
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    # "id"を取得してリストに追加
                    if "id" in data:
                        lines.append(data["id"])
                    else:
                        print(f'"id" が見つかりません: {file_path}')
            except Exception as e:
                print(f"エラーが発生しました: {file_path}, {e}")

# 結果を表示
print("取得したIDリスト:", lines)


取得したIDリスト: ['3fa6a05b62169e0d2350d07acbcc057a', '2a3d485b0214d6a182389daa2190d234', '3fe8243b660d5e8fbc0909d98a1ff2b4', '3a3827f1a76c5230e24527abcb488f31', '1bba3fb413b93890947bbeb9022263b8', '4a6961377146f08cb45fd154bd3b6957', '4a199b1c3c80c8823bfdf036d38c839b', '3badd7765618bd66a532a2f6f060af39', '1e8adc1e176bc68571d03b466c72ce41', '3fe365251b54087af0478431b5ad57db', '1c2e9dedbcf511e616a077c4c0fc1181', '2d4a57a467307d675e9e2656aff7dd5b', '2e468cc6afe2da70bc0909d98a1ff2b4', '1f672d2fd5e3f4e78026abe712c1ab05', '1bdeb4aaa0aaea4b4f95630cc18536e0', '3c98bc293d33375a9a6e43b878d5b335', '1caa02b831cccff090baeef8ba5b93e5', '4c5b8babafbb7b5f937ae00fead8910d', '4ae3924f14d2c0084c3a35cee92bb95b', '3c7e4628a9ea201bbf3143b1cb6076a', '2d9a7863dcfb5663d208f79049825a82', '1f08b579e153b2de313f9af5275b7c70', '1a963a929d9b1332290d63dca780cfb6', '1e358e70c047687a1a8831797284b67', '2e9c1c3d866c4b6ce2c6d8aa34a5d35b', '3a6d6534045b1895e8ed194c80e0b1ef', '3f80ce1461f3dbfe16af5d7a0b735543', '4afcc2cf695baea99

In [ ]:
# @title temp
# import shutil

# # 削除したいフォルダのパスを指定
# folder_path = "/content/extracted_folder"

# # フォルダを削除
# try:
#     shutil.rmtree(folder_path)
#     print(f"フォルダ {folder_path} を削除しました。")
# except FileNotFoundError:
#     print(f"フォルダ {folder_path} は存在しません。")
# except Exception as e:
#     print(f"エラーが発生しました: {e}")


In [ ]:
# @title Load 3Dmodel
import trimesh
import numpy as np
import torch

def load_obj_with_colors(obj_path, mtl_path=None, model_id=None):
    """
    Load an OBJ file along with its colors, if available.

    Parameters:
    - obj_path (str): Path to the .obj file.
    - mtl_path (str, optional): Path to the .mtl file (if any).

    Returns:
    - verts (torch.Tensor): Tensor of vertices (Nx3).
    - faces (torch.Tensor): Tensor of faces (Mx3).
    - R, G, B (torch.Tensor): Tensors of vertex colors (N,).
    """
    # Load the OBJ file using trimesh
    mesh = trimesh.load_mesh(obj_path, process=False)

    # Get vertices (verts)
    verts = torch.tensor(mesh.vertices, dtype=torch.float32)

    # Get faces (faces)
    faces = torch.tensor(mesh.faces, dtype=torch.long)

    # Initialize default colors as white
    R = G = B = torch.ones(len(verts), dtype=torch.float32)

    try:
        # Handle materials if present
        if isinstance(mesh.visual, trimesh.visual.TextureVisuals):
            # If texture image exists
            if mesh.visual.material.image is not None:
                uv = mesh.visual.uv  # UV mapping coordinates
                texture_image = np.array(mesh.visual.material.image)  # Texture as a numpy array

                # Map UV coordinates to pixel locations
                u = (uv[:, 0] * (texture_image.shape[1] - 1)).astype(int)
                v = ((1 - uv[:, 1]) * (texture_image.shape[0] - 1)).astype(int)
                colors = texture_image[v, u, :3]  # Extract RGB values

                # Convert to torch tensors
                R, G, B = [torch.tensor(colors[:, i], dtype=torch.float32) for i in range(3)]
            elif mesh.visual.vertex_colors is not None:
                # If vertex colors are directly available
                colors = mesh.visual.vertex_colors[:, :3]  # RGBA to RGB
                R, G, B = [torch.tensor(colors[:, i], dtype=torch.float32) for i in range(3)]
        elif hasattr(mesh.visual.material, "Kd"):
            # Single material color (diffuse reflectance)
            color = np.array(mesh.visual.material.Kd)  # Extract RGB
            R = torch.full((len(verts),), color[0], dtype=torch.float32)
            G = torch.full((len(verts),), color[1], dtype=torch.float32)
            B = torch.full((len(verts),), color[2], dtype=torch.float32)
    except Exception as e:
        print(f"Error while processing colors: {e}")
        error_models.append(model_id)
    #0～1にスケーリング
    R = R/255
    G = G/255
    B = B/255
    return verts, faces, R, G, B


#load model
model_verts = []
model_faces = []
model_R = []
model_G = []
model_B = []
error_models = []
nocaption_models = []

for i, line in enumerate(lines):
  directory_path = f"/content/extracted_folder/{file_name}/{line}/"

  # "screenshots" フォルダが存在するか確認
  screenshots_path = os.path.join(directory_path, "screenshots")

  if os.path.exists(screenshots_path) and os.path.isdir(screenshots_path):
      #print(f"フォルダ '{screenshots_path}' が存在します。処理を実行します。")
      # 必要な処理をここに記述
      obj_path = f"/content/extracted_folder/{file_name}/{line}/models/model_normalized.obj"
      mtl_path = f"/content/extracted_folder/{file_name}/{line}/models/model_normalized.mtl"
      verts, faces, R, G, B = load_obj_with_colors(obj_path, mtl_path,line)
      model_verts.append(verts)
      model_faces.append(faces)
      model_R.append(R)
      model_G.append(G)
      model_B.append(B)
      #print("loading",line)

  else:
      #print(f"フォルダ '{screenshots_path}' は存在しません。")
      nocaption_models.append(line)


# 結果の確認
print("Vertices:", len(model_verts))
print("Faces:", len(model_faces))
print("R:", len(model_R))
print("G:", len(model_G))
print("B:", len(model_B))
print("error_models:",error_models)
print("nocaption",nocaption_models)


Vertices: 100
Faces: 100
R: 100
G: 100
B: 100
error_models: []
nocaption []


#####Dataset and optimizer

In [ ]:
# @title LoadDataset
TensorName = "output_mesh.npy" # @param {"type":"string","placeholder":"TensorFileName"}
TrainingData = 30 # @param  {"type":"number","placeholder":"default;50"}
ValidationData = 10 # @param  {"type":"number","placeholder":"default;10"}
batch_size = 1 # @param  {"type":"number","placeholder":"default;10"}

#Dataset_Path = "lambdalabs/naruto-blip-captions" # @param {"type":"string","placeholder":"TensorFileName"}
#Input_Name = "text" # @param {"type":"string","placeholder":"TensorFileName"}
#Output_Name = "image" # @param {"type":"string","placeholder":"TensorFileName"}

#frame_size = 128 # @param {"type":"number","placeholder":"framesize"}

import torch
import PIL
from typing import List, Union
import numpy as np
from datasets import Dataset, DatasetDict

def pil_to_numpy(images: Union[List[PIL.Image.Image], PIL.Image.Image]) -> np.ndarray:
    """
    Convert a PIL image or a list of PIL images to NumPy arrays.

    Args:
        images (`PIL.Image.Image` or `List[PIL.Image.Image]`):
            The PIL image or list of images to convert to NumPy format.

    Returns:
        `np.ndarray`:
            A NumPy array representation of the images.
    """
    if not isinstance(images, list):
        images = [images]
    images = [np.array(image).astype(np.float32) / 255.0 for image in images]
    images = np.stack(images, axis=0)

    return images

# loading dataset
loaded_numpy = np.load(TensorName)
loaded_numpy = loaded_numpy.astype(np.float32)
latent =  torch.from_numpy(loaded_numpy).clone()

model_verts = model_verts[:len(latent)]
model_faces = model_faces[:len(latent)]
model_R = model_R[:len(latent)]
model_G = model_G[:len(latent)]
model_B = model_B[:len(latent)]


# リストをテンソルに変換
# 各リスト内のテンソルをclone().detach()で再構築
model_verts_tensor = [v.clone().detach() for v in model_verts]
model_faces_tensor = [f.clone().detach() for f in model_faces]
model_R_tensor = [r.clone().detach() for r in model_R]
model_G_tensor = [g.clone().detach() for g in model_G]
model_B_tensor = [b.clone().detach() for b in model_B]


# 各テンソルをスタック
features = {
    "latent": latent,
    "verts": model_verts_tensor,
    "faces": model_faces_tensor,
    "R": model_R_tensor,
    "G": model_G_tensor,
    "B": model_B_tensor
}



# Dataset を作成
original_dataset = Dataset.from_dict(features)

# train, test 用に分割
dataset = DatasetDict({
    "train": original_dataset.select(range(0, TrainingData)),  # 0〜49番目を訓練用
    "validation": original_dataset.select(range(TrainingData, TrainingData+ValidationData)), # 50〜をテスト用
})


def convert_input_to_tensor(example):
    example["latent"] = torch.tensor(example[Input_Name])  # Input_Name を tensor に変換
    return example

# Output_Name を numpy.ndarray に変換
def convert_output_to_numpy(example):
    example["model"] = np.array(example[Output_Name])  # Output_Name を numpy 配列に変換
    return example

print(type(dataset["train"]["latent"][0]))
print(type(dataset["train"]["verts"][0]))



<class 'list'>
<class 'list'>


In [ ]:
# @title Dataloader
from torchvision import transforms
# from transformers import CLIPTokenizer
# # data preprocessing


# def custom_collate_fn(batch):
#     latents = [item["latent"] for item in batch]    # テキストを取り出す
#     models = [item["model"] for item in batch]  # 画像を取り出す

#     #print(texts[0].shape)
#     return {"latent": latents,"model": models}


train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)

eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
    )

In [ ]:
# @title optimizer and lr scheduler
lr = 2e-5 # @param {"type":"number","placeholder":"1e-3"}
num_epochs = 85 # @param {"type":"slider","min":1,"max":100,"step":1}
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.shap_e_renderer.parameters()),
    lr=lr
    )

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
# @title clear chache
import gc
gc.collect()
!free -h

               total        used        free      shared  buff/cache   available
Mem:            12Gi       7.0Gi       4.2Gi       1.0Mi       1.5Gi       5.4Gi
Swap:             0B          0B          0B


#####Training

In [ ]:
# @title train
grid_size = 64  #@param {"type":"number","placeholder":"2^n,ex)64,128"}

import torch
import torch.nn.functional as F
from PIL import Image
import types
#import torch_xla.core.xla_model as xm
import random
# training and evaluation
from typing import Dict, Optional, Tuple
import importlib
from tqdm.notebook import tqdm
import fpsample



# def farthest_point_sampling(points, num_samples):
#     """
#     Farthest Point Sampling (FPS)を手動で実装する。

#     Args:
#     - points (torch.Tensor): 点群の座標、形状は [N, 3] のテンソル。
#     - num_samples (int): サンプリングする点の数。

#     Returns:
#     - torch.Tensor: サンプリングされた点のインデックス（形状は [num_samples]）。
#     """
#     N = points.shape[0]
#     selected_indices = []

#     # 最初の点をランダムに選ぶ
#     selected_indices.append(np.random.randint(0, N))

#     # 最も遠い点を選んでいく
#     dist = torch.full((N,), float('inf'))  # 初期状態では全ての点との距離を無限大に設定

#     for _ in range(1, num_samples):
#         for i in range(N):
#             dist[i] = min(dist[i], torch.norm(points[i] - points[selected_indices[-1]]))

#         # 最も遠い点を選ぶ
#         farthest_point_idx = torch.argmax(dist).item()
#         selected_indices.append(farthest_point_idx)

#     return torch.tensor(selected_indices)

# 新しい関数をインスタンスメソッドとしてセット
# model.shap_e_renderer.decode_to_mesh = new_decode_to_mesh

# class ShapERenderer:
#
#     def decode_to_mesh(self):
#         # 新しいメソッドの実装
#         pass

# class model.shap_e_renderer:
#   def decode_to_mesh
        # pass

model = model.to(device)

train_losses = []
eval_losses = []
train_ppls = []
eval_ppls = []


for epoch in tqdm(range(num_epochs)):
    model.shap_e_renderer.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {
            "latent": batch["latent"],
            "verts": batch["verts"],
            "faces": batch["faces"],
            "R" : batch["R"],
            "G" : batch["G"],
            "B" : batch["B"]
        }
        # "latent": latent,
        # "verts": model_verts_tensor,
        # "faces": model_faces_tensor,
        # "R": model_R_tensor,
        # "G": model_G_tensor,
        # "B": model_B_tensor
        #print(len(batch["text"]))
        # loaded_tensor = torch.from_numpy(batch["text"]).clone()
        # latent = loaded_tensor.to(device)
        latents = batch["latent"][0].to(device).requires_grad_(True)
        # print("latent",latents.requires_grad)
        # print(latents.shape)
        torch.cuda.empty_cache()  # メモリのキャッシュを解放
        outputs = []
        for i, latent in enumerate(latents):
            output = model.shap_e_renderer.new_mesh(
                latent[None, :],
                device,
                grid_size = grid_size
            )
            outputs.append(output)

        batch_R = batch["R"][0].to(device)
        batch_G = batch["G"][0].to(device)
        batch_B = batch["B"][0].to(device)
        torch.cuda.empty_cache()

        # print(batch["verts"][0].shape[0])#check
        # print(outputs[0].verts.shape[0])#check
        # vertices2のインデックスに基づいて色をリサンプリング
        colors2_combined = torch.stack((batch_R, batch_G, batch_B), dim=-1)
        if batch["verts"][0].shape[0] >= outputs[0].verts.shape[0]:
            idx = fpsample . bucket_fps_kdline_sampling ( batch["verts"][0] ,  outputs[0].verts.shape[0] ,  h = 3 )
            colors2_combined = colors2_combined[idx]
            sampled_R = colors2_combined[:, 0].to(device)
            sampled_G = colors2_combined[:, 1].to(device)
            sampled_B = colors2_combined[:, 2].to(device)
            # vertices2もインデックスに基づいてリサンプリング
            sampled_vertices2 = batch["verts"][0][idx].to(device)

        else:
            # print("else")#check
            idx = fpsample . bucket_fps_kdline_sampling ( outputs[0].verts.detach().cpu() , batch["verts"][0].shape[0]  ,  h = 3 )
            outputs[0].verts = outputs[0].verts[idx]
            outputs[0].vertex_channels['R'] = outputs[0].vertex_channels['R'][idx]
            outputs[0].vertex_channels['G'] = outputs[0].vertex_channels['G'][idx]
            outputs[0].vertex_channels['B'] = outputs[0].vertex_channels['B'][idx]
            sampled_vertices2 = batch["verts"][0]
            sampled_R = batch_R
            sampled_G = batch_G
            sampled_B = batch_B

        # print(sampled_R.shape)
        # print(sampled_vertices2.shape)


        losses = []
        #loss = torch.tensor([], dtype=torch.float32,  device = device)
        loss = torch.tensor(0.0, device=device)
        # MSE損失を計算 平均二乗誤差
        #print("outputverts",outputs[0].verts.requires_grad)

        loss += F.mse_loss(outputs[0].verts.clone(), sampled_vertices2)
        losses.append(F.mse_loss(outputs[0].verts.clone(), sampled_vertices2))
        # print("verts",loss.requires_grad)
        #print("outputfaces",outputs[0].faces.requires_grad)
        # outputs[0].faces = interpolate_mesh_vertices(outputs[0].faces, batch["faces"][0].shape[0])
        # loss += F.mse_loss(outputs[0].faces.clone().float(), batch["faces"][0].to(device).float())
        #print("faces",loss.requires_grad)

        #rint("outputR:",outputs[0].vertex_channels["R"].requires_grad)
        loss += F.mse_loss(outputs[0].vertex_channels['R'].clone(), sampled_R)
        losses.append(F.mse_loss(outputs[0].vertex_channels['R'].clone(), sampled_R))
        # print(loss.requires_grad)

        #print("outputG:",outputs[0].vertex_channels["G"].requires_grad)
        loss += F.mse_loss(outputs[0].vertex_channels['G'].clone(), sampled_G)
        losses.append(F.mse_loss(outputs[0].vertex_channels['G'].clone(), sampled_G))
        # print(loss.requires_grad)

        #print("outputB:",outputs[0].vertex_channels["B"].requires_grad)
        loss += F.mse_loss(outputs[0].vertex_channels['B'].clone(), sampled_B)
        losses.append(F.mse_loss(outputs[0].vertex_channels['B'].clone(), sampled_B))
        #print("loss_grad:",loss.requires_grad)

        # #loss = torch.mean(torch.stack(loss))
        # # print(f"Calculated loss: {loss.item()}")
        total_loss += loss.detach().float()
        #!free -h

        torch.autograd.set_detect_anomaly(True)
        loss.backward()

        # lora_params = [param for name, param in model.shap_e_renderer.named_parameters() if param.requires_grad]

        #         # 勾配計算
        # grads = torch.autograd.grad(
        #     outputs=loss,
        #     inputs=lora_params,
        #     retain_graph=True,
        #     create_graph=False,
        #     allow_unused=True
        # )

        # # 勾配を .grad に設定
        # for param, grad in zip(lora_params, grads):
        #     param.grad = grad

        # for name, param in model.shap_e_renderer.named_parameters():
        #     print(f"requires_grad for {name}: {param.requires_grad}")

        # for name, param in model.shap_e_renderer.named_parameters():
        #     if param.requires_grad:
        #         print(f"Gradients for {name}: {param.grad}")

        # for name, param in model.shap_e_renderer.named_parameters():
        #     if param.requires_grad:
        #         print(f"Before update {name}: {param.data}")

        optimizer.step()

        # for name, param in model.shap_e_renderer.named_parameters():
        #     if param.requires_grad:
        #         print(f"After update {name}: {param.data}")

        lr_scheduler.step()
        optimizer.zero_grad()
        # if step % log_interval == 0:
        print(f"Step [{step}/{len(train_dataloader)}], totalLoss: {total_loss.item()}, Calculated loss: {loss.item()}")

    model.shap_e_renderer.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {
            "latent": batch["latent"],
            "verts": batch["verts"],
            "faces": batch["faces"],
            "R" : batch["R"],
            "G" : batch["G"],
            "B" : batch["B"]
        }
        latents = batch["latent"][0].to(device)
        #print(latents.shape)
        outputs = []
        for i, latent in enumerate(latents):
            output = model.shap_e_renderer.decode_to_mesh(
            latent[None, :],
            device,
            )
            outputs.append(output)
        # images = torch.stack(images)
        # images = images.cpu().numpy()
        # images = images[0][random.randint(0, 19)]




        batch_R = batch["R"][0].to(device)
        batch_G = batch["G"][0].to(device)
        batch_B = batch["B"][0].to(device)
        torch.cuda.empty_cache()

        # print(batch["verts"][0].shape[0])
        # print(outputs[0].verts.shape[0])
        # vertices2のインデックスに基づいて色をリサンプリング
        colors2_combined = torch.stack((batch_R, batch_G, batch_B), dim=-1)
        if batch["verts"][0].shape[0] >= outputs[0].verts.shape[0]:
            idx = fpsample . bucket_fps_kdline_sampling ( batch["verts"][0] ,  outputs[0].verts.shape[0] ,  h = 3 )
            colors2_combined = colors2_combined[idx]
            sampled_R = colors2_combined[:, 0].to(device)
            sampled_G = colors2_combined[:, 1].to(device)
            sampled_B = colors2_combined[:, 2].to(device)
            # vertices2もインデックスに基づいてリサンプリング
            sampled_vertices2 = batch["verts"][0][idx].to(device)

        else:
            # print("else")#check
            idx = fpsample . bucket_fps_kdline_sampling ( outputs[0].verts.detach().cpu() , batch["verts"][0].shape[0]  ,  h = 3 )
            outputs[0].verts = outputs[0].verts[idx]
            outputs[0].vertex_channels['R'] = outputs[0].vertex_channels['R'][idx]
            outputs[0].vertex_channels['G'] = outputs[0].vertex_channels['G'][idx]
            outputs[0].vertex_channels['B'] = outputs[0].vertex_channels['B'][idx]
            sampled_vertices2 = batch["verts"][0]
            sampled_R = batch_R
            sampled_G = batch_G
            sampled_B = batch_B




        losses = []
        #loss = torch.tensor([], dtype=torch.float32,  device = device)
        loss = torch.tensor(0.0, device=device)
        # MSE損失を計算 平均二乗誤差
        #print("outputverts",outputs[0].verts.requires_grad)

        loss += F.mse_loss(outputs[0].verts.clone(), sampled_vertices2.to(device))
        losses.append(F.mse_loss(outputs[0].verts.clone(), sampled_vertices2.to(device)))
        # print("verts",loss.requires_grad)
        #print("outputfaces",outputs[0].faces.requires_grad)
        # outputs[0].faces = interpolate_mesh_vertices(outputs[0].faces, batch["faces"][0].shape[0])
        # loss += F.mse_loss(outputs[0].faces.clone().float(), batch["faces"][0].to(device).float())
        #print("faces",loss.requires_grad)

        #rint("outputR:",outputs[0].vertex_channels["R"].requires_grad)
        loss += F.mse_loss(outputs[0].vertex_channels['R'].clone(), sampled_R.to(device))
        losses.append(F.mse_loss(outputs[0].vertex_channels['R'].clone(), sampled_R.to(device)))
        # print(loss.requires_grad)

        #print("outputG:",outputs[0].vertex_channels["G"].requires_grad)
        loss += F.mse_loss(outputs[0].vertex_channels['G'].clone(), sampled_G.to(device))
        losses.append(F.mse_loss(outputs[0].vertex_channels['G'].clone(), sampled_G.to(device)))
        # print(loss.requires_grad)

        #print("outputB:",outputs[0].vertex_channels["B"].requires_grad)
        loss += F.mse_loss(outputs[0].vertex_channels['B'].clone(), sampled_B.to(device))
        losses.append(F.mse_loss(outputs[0].vertex_channels['B'].clone(), sampled_B.to(device)))
        #print("loss_grad:",loss.requires_grad)

        eval_loss += loss.detach().float()


    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=}")
    print(f"{epoch=}: {eval_ppl=} {eval_epoch_loss=}")
    train_losses.append(train_epoch_loss)
    eval_losses.append(eval_epoch_loss)
    train_ppls.append(train_ppl)
    eval_ppls.append(eval_ppl)

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Step [0/30], totalLoss: 1.157444953918457, Calculated loss: 1.157444953918457
Step [1/30], totalLoss: 1.8990663290023804, Calculated loss: 0.7416213750839233
Step [2/30], totalLoss: 2.6783792972564697, Calculated loss: 0.7793130278587341
Step [3/30], totalLoss: 3.5669994354248047, Calculated loss: 0.8886200189590454
Step [4/30], totalLoss: 4.673511505126953, Calculated loss: 1.1065120697021484
Step [5/30], totalLoss: 5.312392711639404, Calculated loss: 0.638881266117096
Step [6/30], totalLoss: 6.255434036254883, Calculated loss: 0.9430414438247681
Step [7/30], totalLoss: 7.010824203491211, Calculated loss: 0.755389928817749
Step [8/30], totalLoss: 7.436095237731934, Calculated loss: 0.42527082562446594
Step [9/30], totalLoss: 8.595550537109375, Calculated loss: 1.1594549417495728
Step [10/30], totalLoss: 9.096037864685059, Calculated loss: 0.5004875659942627
Step [11/30], totalLoss: 9.953263282775879, Calculated loss: 0.8572255373001099
Step [12/30], totalLoss: 10.715792655944824, Calc

  0%|          | 0/10 [00:00<?, ?it/s]

epoch=0: train_ppl=tensor(2.2098, device='cuda:0') train_epoch_loss=tensor(0.7929, device='cuda:0')
epoch=0: eval_ppl=tensor(2.2215, device='cuda:0') eval_epoch_loss=tensor(0.7982, device='cuda:0')


...

  0%|          | 0/10 [00:00<?, ?it/s]

epoch=73: train_ppl=tensor(1.9019, device='cuda:0') train_epoch_loss=tensor(0.6429, device='cuda:0')
epoch=73: eval_ppl=tensor(1.8821, device='cuda:0') eval_epoch_loss=tensor(0.6324, device='cuda:0')


  0%|          | 0/30 [00:00<?, ?it/s]

Step [0/30], totalLoss: 0.843798816204071, Calculated loss: 0.843798816204071
Step [1/30], totalLoss: 1.396357536315918, Calculated loss: 0.5525587201118469
Step [2/30], totalLoss: 1.896040916442871, Calculated loss: 0.4996834397315979
Step [3/30], totalLoss: 2.39518404006958, Calculated loss: 0.49914318323135376
Step [4/30], totalLoss: 2.977541923522949, Calculated loss: 0.5823578834533691
Step [5/30], totalLoss: 3.6316637992858887, Calculated loss: 0.6541217565536499
Step [6/30], totalLoss: 4.116559028625488, Calculated loss: 0.48489516973495483
Step [7/30], totalLoss: 4.676589012145996, Calculated loss: 0.560029923915863
Step [8/30], totalLoss: 5.688220024108887, Calculated loss: 1.0116312503814697
Step [9/30], totalLoss: 6.507495880126953, Calculated loss: 0.8192758560180664
Step [10/30], totalLoss: 6.9025421142578125, Calculated loss: 0.3950459957122803
Step [11/30], totalLoss: 7.123964786529541, Calculated loss: 0.22142282128334045
Step [12/30], totalLoss: 7.6913161277771, Calcul

RuntimeError: torch.cat(): expected a non-empty list of Tensors

##### last

In [ ]:
print("train",TrainingData)
print("eval",ValidationData)
print("batch",batch_size)
print("epoch",num_epochs)
print("r",r)
print("lora_alpha",lora_alpha)
print("lora_dropout",lora_dropout)
print("lr",lr)
print("eval_losses")
for i in range(num_epochs):
  print(eval_losses[i].item())

print("\neval_ppls")
for i in range(num_epochs):
  print(eval_ppls[i].item())

print("\ntrain_losses")
for i in range(num_epochs):
  print(train_losses[i].item())

print("\ntrain_ppls")
for i in range(num_epochs):
  print(train_ppls[i].item())

In [ ]:
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

nvmlInit()
handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0 を指定
info = nvmlDeviceGetMemoryInfo(handle)

print(f"Total memory: {info.total / 1024 ** 2:.2f} MB")
print(f"Used memory: {info.used / 1024 ** 2:.2f} MB")
print(f"Free memory: {info.free / 1024 ** 2:.2f} MB")


In [ ]:
from diffusers.utils import export_to_gif
from IPython.display import Image
guidance_scale = 15.0
prompt = "a black airplane"
model.to(device)
if info.free/info.total >= 0.5:
    images = model(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=64,
        frame_size=256,
    ).images
    gif_path = export_to_gif(images[0], "test1.gif")
    display(Image(gif_path, format='png'))

prompt = "a airplane with red tail"
if info.free/info.total >= 0.5:
    images = model(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=64,
        frame_size=256,
    ).images
    gif_path = export_to_gif(images[0], "test2.gif")
    display(Image(gif_path, format='png'))

prompt = "a shark"
if info.free/info.total >= 0.5:
    images = model(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=64,
        frame_size=256,
    ).images
    gif_path = export_to_gif(images[0], "test3.gif")
    display(Image(gif_path, format='png'))

prompt = "a airplane"
if info.free/info.total >= 0.5:
    images = model(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=64,
        frame_size=256,
    ).images
    gif_path = export_to_gif(images[0], "test4.gif")
    display(Image(gif_path, format='png'))

prompt = "a airplane"
if info.free/info.total >= 0.5:
    images = model(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=64,
        frame_size=256,
    ).images
    gif_path = export_to_gif(images[0], "test5.gif")
    display(Image(gif_path, format='png'))

In [ ]:
def write_obj_with_colors(filepath, verts, faces, R, G, B):
    """
    Exports a .obj file with vertex colors.

    Args:
        filepath (str): Path to save the .obj file.
        verts (torch.Tensor or np.ndarray): Vertex coordinates, shape (N, 3).
        faces (torch.Tensor or np.ndarray): Face indices, shape (M, 3).
        R, G, B (torch.Tensor or np.ndarray): RGB color components, shape (N,).
    """
    # Ensure input tensors are NumPy arrays
    verts = verts.cpu().numpy() if hasattr(verts, 'cpu') else np.array(verts)
    faces = faces.cpu().numpy() if hasattr(faces, 'cpu') else np.array(faces)
    R = R.cpu().numpy() if hasattr(R, 'cpu') else np.array(R)
    G = G.cpu().numpy() if hasattr(G, 'cpu') else np.array(G)
    B = B.cpu().numpy() if hasattr(B, 'cpu') else np.array(B)

    # Scale RGB values to 255
    R, G, B = R * 255, G * 255, B * 255

    with open(filepath, 'w') as f:
        # Write vertices with colors
        for i in range(len(verts)):
            x, y, z = verts[i]
            r, g, b = R[i], G[i], B[i]
            f.write(f"v {x} {y} {z} {r} {g} {b}\n")

        # Write faces (OBJ format is 1-indexed)
        for i in range(len(faces)):
            v1, v2, v3 = faces[i] + 1
            f.write(f"f {v1} {v2} {v3}\n")

write_obj_with_colors("output.obj", outputs[0].verts, outputs[0].faces, outputs[0].vertex_channels['R'], outputs[0].vertex_channels['G'], outputs[0].vertex_channels['B'])

In [ ]:
# LoRAのパラメータだけを保存
lora_params = {
    name: param for name, param in model.shap_e_renderer.named_parameters() if "lora_" in name
}
torch.save(lora_params, "lora_params.pth")

In [ ]:
from google.colab import files
files.download("test1.gif")
files.download("test2.gif")
files.download("test3.gif")
files.download("test4.gif")
files.download("test5.gif")


##### omake

In [ ]:
# @title savemodel
!mkdir lora
for i in range(6):  # mlp.0 ~ mlp.5 の層をループ
    layer_name_A = f"base_model.model.mlp.mlp.{i}.lora_A.default.weight"
    layer_name_B = f"base_model.model.mlp.mlp.{i}.lora_B.default.weight"

    torch.save(model.shap_e_renderer.state_dict()[layer_name_A], f"/content/lora/mlp_{i}_lora_A_weight.pt")
    torch.save(model.shap_e_renderer.state_dict()[layer_name_B], f"/content/lora/mlp_{i}_lora_B_weight.pt")

In [ ]:
!zip -r lora_param.zip lora

  adding: lora/ (stored 0%)
  adding: lora/mlp_0_lora_A_weight.pt (deflated 16%)
  adding: lora/mlp_2_lora_B_weight.pt (deflated 10%)
  adding: lora/mlp_5_lora_B_weight.pt (deflated 11%)
  adding: lora/mlp_5_lora_A_weight.pt (deflated 11%)
  adding: lora/mlp_3_lora_A_weight.pt (deflated 11%)
  adding: lora/mlp_1_lora_A_weight.pt (deflated 11%)
  adding: lora/mlp_0_lora_B_weight.pt (deflated 10%)
  adding: lora/mlp_2_lora_A_weight.pt (deflated 11%)
  adding: lora/mlp_4_lora_B_weight.pt (deflated 10%)
  adding: lora/mlp_1_lora_B_weight.pt (deflated 10%)
  adding: lora/mlp_3_lora_B_weight.pt (deflated 10%)
  adding: lora/mlp_4_lora_A_weight.pt (deflated 10%)


In [ ]:
del batch
del output,loss,grad
# メモリを解放

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from google.colab import runtime
#runtime.unassign()

#####memo


projを学習しようとするとRAMが切れる